## 数据处理

In [2]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests
from pathlib import Path 
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import _read_text_auto,build_jsonl_for_range, _normalize_date



In [3]:

##QQ的txt文件
pathtxt   = "Peter发言_测试1群(纯净版）.txt"




# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    return_str=False,   # 返回 list[str]
)

print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])


1108 lines
{"发言日期": "2025-07-02", "发言时间": "18:18:01", "玩家ID": "Peter本尊(16186514)", "玩家消息": "@淬 @淬 还真有"}


## 大模型分类

## 定义

In [4]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：对保留内容进行 5 类意图分类（输出带“意图分类”的 JSON 行）
- 结果按 sheet（体感反馈/疑惑不解和问题询问/玩家建议和灵感/情绪输出/问题反馈）写入 Excel
"""
from model_classifyV1_Copy1 import (
    load_system_prompt,build_user_prompt_single_speaker_behavior,extract_valid_json_lines,call_ark_chat_completions
)

## 设置参数

In [5]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点
                 
PROMPT_MD_PATH01 = Path("提示词1.md")      # 模型#1 system 提示词（筛相关）

BATCH_SIZE       = 100
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

## 开始运行

In [6]:
from tqdm import tqdm
import time
system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)


total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：分析（或筛相关）---
        user_prompt1 = build_user_prompt_single_speaker_behavior(batch_lines)
        output_behavior = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=R1_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        print(output_behavior)
        
        # ✅ 修正：检查 output_behavior
        if not output_behavior or not str(output_behavior).strip():
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue

        # ✅ 这里你至少要做：解析 + 写入 + 计数
        # 举例：如果你最终要写入文件/Excel，写入成功后累加
        # write_results(output_behavior)
        written_total += 1  # 先用“批次级计数”兜底，后续可改成“行级计数”

    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)

print("\n✅ 全部批次处理完成！")
print(f"输入总数：{total}")
print(f"写入总数：{written_total}")


准备处理 1108 条，共 12 批（每批 100 条）。


🔥 批处理进度:   0%|                                                                                                                                     | 0/12 [00:00<?, ?批/s]

### 以下是对其表达风格的深度提炼与复刻指南

本报告基于目标人物“Peter本尊(16186514)”在社群中的发言记录（JSONL格式）进行深度分析。输入数据仅包含该人物的发言，未涉及他人互动，因此所有结论均严格引用原句证据，避免虚构上下文。分析聚焦四大面向：表达风格、思维结构、互动策略、叙事节奏，并转化为可复刻方法（包括结构模板、句式槽位和练习句）。证据优先，可观测特征以动作/结构描述；推断部分标记【推断】并说明依据；若依据不足，标注【无法判断】。复刻方法强调句式槽位（如[槽位]）和可操作技巧，避免空泛标签。

---

### 1. 核心人格定位
**核心人格定位：自嘲暴君**  
**核心矛盾：权威开发者 vs 街头混混式自嘲**  
目标人物通过反差制造魅力：一方面以开发者身份建立权威（解释技术约束、规划进度），另一方面用粗俗自嘲（如“老夫”、“素质低”）缓冲冲突，拉近距离。这种矛盾让表达既具压迫感又接地气，易于被接住。  
- **证据原句**：  
  - 「老夫一菜刀劈死你」（展示街头混混式威胁，自嘲“老夫”软化攻击）。  
  - 「我就是素质低啊」（直接自曝缺点，降低他人防御）。  
  - 「从小混社团的」（用过往经历强化自嘲人设，制造亲近感）。  
- **【推断】依据**：发言中频繁切换专业解释（如“开发计划”）与粗俗语言（如“马勒戈壁”），依据是情绪词和句式对比，推测其目的是管理预期（避免用户过高期待）并维持社群控制。但动机细节不足，部分归因【无法判断】。

---

### 2. 表达风格分析
以下支柱提炼自可观测特征（句式、情绪词、论证结构等），每个点解决特定表达目的，并附复刻技巧和例句（用槽位变量如[问题]确保可复制）。复刻技巧聚焦动作顺序：先缓冲情绪（粗俗或自嘲），再加固论点（解释或反问），最后锚定现实。

#### 支柱1: 粗俗缓冲 + 自嘲软化（解决目的：缓冲冲突、拉近距离）  
- **可观测特征**：使用高频粗俗词（如“马勒戈壁”、“我擦”）作为情绪出口，结合自嘲称谓（“老夫”）或自贬（“素质低”）软化攻击性，避免直接对抗。动作结构：粗俗词起头 → 自嘲句缓冲 → 回归主题。  
- **证据原句**：  
  - 「马勒戈壁的，都说了现在半成品」（粗俗起头缓冲不满，自嘲“半成品”降低期待）。  
  - 「我擦，你

🔥 批处理进度:   8%|██████████▎                                                                                                                 | 1/12 [02:19<25:30, 139.14s/批]

### 以下是对其表达风格的深度提炼与复刻指南

通过对目标人物（Peter本尊）在社群中的发言记录进行全盘分析，我提炼出其表达路径的核心特征。所有结论均基于可观测证据（引用原句），避免虚构上下文（输入仅含其发言，无他人互动）。分析聚焦四大面向：表达风格（语言特征）、思维结构（信息组织）、互动策略（对话推进）、叙事节奏（话题流动），并落地到可复刻方法（结构模板、句式槽位、练习句）。复刻核心在于模仿其“表达动作”——如对比论证、自嘲缓冲、直接回应、预期管理——以提升表达的被接住性（易读、易响应、易延续）。

---

### 1. 核心人格定位
**人设命名：自嘲理想主义者**  
**核心矛盾：崇高治愈愿景 vs 现实自嘲坦率**  
该人物以“治愈游戏”为理想核心，却用自嘲、粗俗语言暴露现实缺陷，制造反差魅力。矛盾点在于：一边宣扬光明希望（建立权威），一边自曝“垃圾游戏”或“没心没肺”（拉近距离），形成真实感与幽默缓冲。证据如下：  
- 「这游戏本质上，是我想做一个让世人能够“看见光和希望”的治愈游戏」（展示理想主义）  
- 「蜗牛就一个垃圾氪金游戏，我昧着良心做的」（自嘲现实缺陷）  
- 「我主打一个没心没肺」（强化自嘲人格）  
【推断】动机可能为缓解开发压力或建立社区信任，依据是其频繁自嘲（如“昧着良心”）结合积极结尾（如“不会太监”），可观测到情绪缓冲目的。

---

### 2. 表达风格分析
以下支柱覆盖四大面向，每个点均含可观测特征、证据、复刻技巧及例句（用槽位变量如{主题}、{用户}避免攻击性）。复刻关键动作：优先使用“对比论证”建立深度，接“自嘲缓冲”软化冲突，再“直接@回应”推进对话，最后“预期管理”收尾节奏。

#### 支柱1: 光明-黑暗对比术（覆盖思维结构与表达风格）  
- **解决表达目的**：建立权威深度，让抽象理念易被接住（通过二元对比简化复杂思想）。  
- **可观测特征**：高频使用“光明/黑暗”比喻论证；句式结构为“正因为{负面}，才能{正面}”或“没有A，哪里来B”；情绪词如“治愈”“希望”强化正向。  
- **证据原句**：  
  - 「正因为有黑暗，人们才能看到光明」  
  - 「没有黑暗，哪里来光明」  
  - 「这游戏主打治愈」  
- **复刻技巧**：用槽位模板组织论点——先抛负面槽位制造张

...

### 以下是对其表达风格的深度提炼与复刻指南

本报告基于目标人物“Peter本尊(16186514)”在社群中的发言记录（JSONL输入）进行深度分析。输入数据仅包含该人物的发言，无他人回复或上下文，因此所有结论均严格引用原句证据，避免虚构。分析聚焦四大面向：表达风格、思维结构、互动策略、叙事节奏，并转化为可复刻方法（包括结构模板、句式槽位和练习句）。证据优先，可观测特征（如句式、情绪词、话题切换）直接描述；推断部分标记为【推断】，依据不足则标注【无法判断】。

---

### 1. 核心人格定位
**核心人格定位：创意狂想者与自黑实干家的矛盾体**  
**核心矛盾：天马行空的游戏权威 vs 接地气的自嘲幽默**  
该人物以游戏开发者身份展示专业权威，却通过自黑和夸张化解严肃，制造亲近感与魅力。证据显示，他频繁用个人状态或荒诞比喻缓冲游戏解释，形成“高概念低姿态”的反差。  
- 证据原句：  
  - 「我做这游戏主打一个道心通达」（展示权威理念）。  
  - 「我现在逐步减少药量，每天脑子嗡嗡的」（自曝弱点，增加真实感）。  
  - 「我发起疯来连狗都害怕」（夸张自嘲，软化权威）。  
- 【推断】动机：可能旨在降低用户防御，促进社群参与（依据：发言中多次自嘲后紧跟游戏说明，如自曝“分魂废物”后解释科技球分类）。

---

### 2. 表达风格分析
以下分析点覆盖四大面向（风格、结构、互动、节奏），每个点均基于可观测特征（句式、情绪词、话题切换等），并附复刻技巧与例句（使用槽位如[用户名]、[主题]避免攻击性）。复刻方法聚焦“动作”，而非空泛形容词。

#### 支柱1：互动钩子——@用户 + 即时切入  
- **解决表达目的**：拉近距离，制造参与感，快速启动对话。避免单向输出，确保消息被“接住”。  
- **可观测特征**：发言常以“@用户 @用户”开头（重复@强化提醒），后接直接消息（无寒暄）。结构为“呼叫动作 → 核心内容”，形成高效互动入口。  
- **证据原句**：  
  - 「@楠枫 @楠枫 之后每次重启，会随机生成一个存档」（重复@后立即解释机制）。  
  - 「@炎殇魔尊燯弑天 @炎殇魔尊燯弑天 都说了这是保洁阿姨随手定的」（@后直接反驳）。  
  - 「@枫火 你这种宝可梦大师」（@后个性化调侃）。  
- **

🔥 批处理进度:  92%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊          | 11/12 [20:51<01:47, 107.29s/批]

### 以下是对其表达风格的深度提炼与复刻指南

本报告基于目标人物（玩家ID: Peter本尊(16186514)）在社群中的发言记录（JSONL格式）进行深度分析。输入数据仅包含该人物的发言，无他人回复，因此所有结论均源于可观测的语言特征（如句式、情绪词、论证结构、号召动作、话题切换、重复强调），避免虚构上下文。分析聚焦四大面向（表达风格、思维结构、互动策略、叙事节奏），并落地到可复刻方法（结构模板、句式槽位、练习句）。所有推断均标记【推断】并给出依据；若证据不足，则标注【无法判断】。证据引用保留原句，确保真实性。

---

### 1. 核心人格定位
目标人物的表达核心在于“专业压力下的戏谑自保”，通过反差制造亲近感与权威缓冲。核心矛盾体现为“严肃开发者身份 vs 荒诞自黑幽默”：在讨论专业议题（如游戏开发进度）时，频繁插入粗口、夸张自嘲或突发玩笑，以此化解冲突、拉近距离，而非纯粹抱怨。证据显示，该人物常用自我贬低或虚构场景（如“护工换班”）来管理预期，避免直接对抗。动机可能源于压力释放【推断】，依据是发言密集围绕开发挫折（如版本错误、进度延迟），随后总伴随自嘲或 abrupt 退出，形成情绪安全阀。

- **人设命名**：戏谑压力阀  
- **核心矛盾**：专业严谨 vs 荒诞自保  
- **证据**：  
  - 「护工换班，我就出来了」——虚构场景自嘲，缓冲工作失误。  
  - 「我要自己把束缚带穿上」——夸张自我约束，回应潜在批评。  
  - 「道爷我成了~」——戏谑成功宣言，淡化长期压力。  

---

### 2. 表达风格分析
以下支柱基于可观测特征（非空泛形容词），提炼目标人物如何组织信息、推进互动、管理节奏。每个支柱解决特定表达目的（如缓冲冲突、快速退出），并附复刻技巧（具体方法）和复刻例句（句式槽位：用[变量]占位，便于替换）。证据来自至少3条原句，确保可复现。

#### 支柱1: 粗口强化吐槽——解决表达目的：释放挫折感，制造张力以吸引注意  
- **可观测特征**：在问题描述中插入高频粗口词（如“特么”、“鸡儿”、“老子”），形成情绪爆发点，让表达更具冲击力，易被读完。结构常为“[问题主题] + [粗口词] + [后果]”，话题切换快速，避免冗长。  
- **证据**：  
  - 「马勒戈壁，国庆前竟然更新版本了」

🔥 批处理进度: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [22:47<00:00, 113.94s/批]


✅ 全部批次处理完成！
输入总数：1108
写入总数：12


In [13]:
print(output_behavior)

### 一、关键节点拆解（按时间）

#### 1. 节点#1 时间：2025-07-16 17:55:50
- 触发线索：刚睡醒听到某人发言
- 立场句：强烈反对（“马勒戈壁”）
- 论证结构：情绪宣泄（“一睡醒就听这货bbbbb”），开发周期解释（“几周时间能把游戏做完”）
- 情绪能量：强度3（愤怒/爆发）
- 行动导向：无具体行动引导，纯情绪输出
【证据】：“你当老夫神仙咩”“就你懂完了”

#### 2. 节点#2 时间：2025-07-16 18:10:01
- 触发线索：测试目的自述
- 立场句：中立观察（“来测试的目的就是看看你们在聊些啥”）
- 论证结构：目标陈述（收集反馈）、筛选标准（“说话讨厌的人踢掉”）
- 情绪能量：强度1（冷静）
- 行动导向：隐含行动指令（通过发言筛选用户）
【证据】：“然后我把说话讨厌的人踢掉，把没那么讨厌的问题修掉”

#### 3. 节点#3 时间：2025-07-16 20:27:14
- 触发线索：游戏核心理念阐述
- 立场句：价值主张（“治愈游戏”）
- 论证结构：对比论证（黑暗/光明）、情感共鸣（“小猫咪照顾你”）
- 情绪能量：强度2（积极/感染）
- 行动导向：无直接行动，但构建情感连接
【证据】：“这游戏本质上，是我想做一个让世人能够‘看见光和希望’的治愈游戏”

#### 4. 节点#4 时间：2025-09-12 16:56:13
- 触发线索：开发调研经历
- 立场句：支持真实性（“缅北实地考察”）
- 论证结构：亲历证据（“增加创作体感”）、夸张表述（“绿水青山”）
- 情绪能量：强度1（调侃）
- 行动导向：无
【证据】：“我特地去缅北实地考察了一圈”“这边绿水青山”

---

### 二、表达路径模板

#### 模板1：情绪爆发式反驳
- 结构链路：触发事件→情绪词爆发→开发解释→身份强调（“老夫”）→威胁/调侃收尾
- 句式槽位：  
  “{情绪词}！{对方观点}→{开发事实}→{身份标榜}→{收尾威胁}”  
  例：“马勒戈壁！就你懂完了？2周时间能做鸡毛内容→老夫一菜刀劈死你”
- 风险点：易引战，需配合社群身份背书

#### 模板2：理念升华式表达
- 结构链路：抛出矛盾→对比框架→情感锚点（动物/孩子）→价值宣言  
- 句式槽位：  
  “没有{负面}，哪里来{正面